# GRU4REC 

In [ ]:
!pip install transformers4rec[pytorch,nvtabular]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 KB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 KB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import glob

import torch 
import transformers4rec.torch as tr

from transformers4rec.torch.ranking_metric import NDCGAt, RecallAt
from transformers4rec.torch.utils.examples_utils import wipe_memory

In [ ]:
from merlin_standard_lib import Schema

# Define schema object to pass it to the TabularSequenceFeatures class
SCHEMA_PATH = '/content/drive/MyDrive/dataset_rees46/processed_nvt/schema.pbtxt'
schema = Schema().from_proto_text(SCHEMA_PATH)

# Create a sub-schema only with the selected features
schema = schema.select_by_name(['product_id-list_seq'])

In [ ]:
schema

[{'name': 'product_id-list_seq', 'value_count': {'min': '2', 'max': '20'}, 'type': 'INT', 'int_domain': {'name': 'product_id', 'max': '166795', 'is_categorical': True}, 'annotation': {'tag': ['list', 'item_id', 'id', 'categorical', 'item'], 'comment': ['{"is_ragged": true, "embedding_sizes": {"dimension": 512.0, "cardinality": 166796.0}, "is_list": true, "freq_threshold": 0.0, "dtype_item_size": 64.0, "start_index": 1.0, "cat_path": ".//categories/unique.product_id.parquet", "max_size": 0.0, "num_buckets": null}']}}]

In [ ]:
sequence_length = 20
inputs = tr.TabularSequenceFeatures.from_schema(
        schema,
        max_sequence_length= sequence_length,
        masking = 'causal',
    )

In [ ]:
d_model = 128
body = tr.SequentialBlock(
        inputs, 
        tr.MLPBlock([d_model]), #projection MLP layer
        tr.Block(torch.nn.GRU(input_size=d_model, 
                              hidden_size=d_model, 
                              num_layers=1, 
                              dropout=0.0, #regularization
                              ), \
                 [None, 20, d_model]) #GRU model
)

In [ ]:
head = tr.Head(
    body,
    tr.NextItemPredictionTask(weight_tying=True, 
                              metrics=[NDCGAt(top_ks=[10, 20], labels_onehot=True),  
                                       RecallAt(top_ks=[10, 20], labels_onehot=True)]),
)

model = tr.Model(head)

In [ ]:
# import NVTabular dependencies
from transformers4rec.torch.utils.data_utils import MerlinDataLoader

x_cat_names, x_cont_names = ['product_id-list_seq'], [] #categorical and continuous features

# dictionary representing max sequence length for each column - needed in the dataloader function
# [column_name: integer] as [key: value]
sparse_features_max = {
    fname: sequence_length for fname in x_cat_names + x_cont_names
}

# Define a `get_dataloader` function to call in the training loop
def get_dataloader(path, batch_size=32):

    return MerlinDataLoader.from_schema(
        schema, 
        path, 
        batch_size, #number of samples to yield at each iteration
        max_sequence_length=sequence_length, 
        sparse_names=x_cat_names + x_cont_names, #list with column names of columns that should be represented as sparse tensors
        sparse_max=sparse_features_max, #dictionary of [column_name: integer] as [key: value], representing max sequence lenght for each column
)
    

In [ ]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer

#Set arguments for training 
train_args = T4RecTrainingArguments(local_rank = -1, 
                                    dataloader_drop_last = False,
                                    report_to = [],   #set empty list to avoid logging metrics to Weights&Biases
                                    gradient_accumulation_steps = 1,
                                    per_device_train_batch_size = 448, 
                                    per_device_eval_batch_size = 512,
                                    output_dir = "./gru", 
                                    max_sequence_length=sequence_length,
                                    learning_rate=0.0007107976722774954,
                                    num_train_epochs=10,
                                    logging_steps=200,
                                    weight_decay=4.0070030423993165e-06,         
                                   )

In [ ]:
# Instantiate the T4Rec Trainer, which manages training and evaluation
trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema,
    compute_metrics=True,
)

In [ ]:
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", "/content/drive/MyDrive/dataset_rees46/sessions_by_day")

In [ ]:
import time


start_time_window_index = 1
final_time_window_index = 30
for time_index in range(start_time_window_index, final_time_window_index):
    # Set data 
    time_index_train = time_index
    time_index_eval = time_index + 1
    train_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_train}/train.parquet"))
    eval_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_eval}/test.parquet"))
    
    # Initialize dataloaders
    trainer.train_dataloader = get_dataloader(train_paths, train_args.per_device_train_batch_size)
    trainer.eval_dataloader = get_dataloader(eval_paths, train_args.per_device_eval_batch_size)
    
    # Train on day related to time_index 
    print('*'*20)
    print("Launch training for day %s are:" %time_index)
    print('*'*20 + '\n')
    trainer.reset_lr_scheduler()
    trainer.train()
    trainer.state.global_step +=1
    
    # Evaluate on the following day
    train_metrics = trainer.evaluate(metric_key_prefix='eval')
    print('*'*20)
    print("Eval results for day %s are:\t" %time_index_eval)
    print('\n' + '*'*20 + '\n')
    for key in sorted(train_metrics.keys()):
        print(" %s = %s" % (key, str(train_metrics[key]))) 
    wipe_memory()
    time.sleep(10)

/usr/local/lib/python3.8/dist-packages/merlin/io/dataset.py:253: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 112448
  Num Epochs = 10
  Instantaneous batch size per device = 448
  Total train batch size (w. parallel, distributed & accumulation) = 448
  Gradient Accumulation steps = 1
  Total optimization steps = 2510


********************
Launch training for day 1 are:
********************



Step,Training Loss
200,10.030400
400,9.033600
600,8.833500
800,8.672500
1000,8.512100
1200,8.359900
1400,8.242200
1600,8.164000
1800,8.090000
2000,8.029400


Saving model checkpoint to ./gru/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./gru/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./gru/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./gru/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./gru/checkpoint-2500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




********************
Eval results for day 2 are:	

********************

 eval_/loss = 8.556951522827148
 eval_/next-item/ndcg_at_10 = 0.06896374374628067
 eval_/next-item/ndcg_at_20 = 0.08415067195892334
 eval_/next-item/recall_at_10 = 0.12966495752334595
 eval_/next-item/recall_at_20 = 0.1898038536310196
 eval_runtime = 2.2688
 eval_samples_per_second = 5867.406
 eval_steps_per_second = 11.46


...

/usr/local/lib/python3.8/dist-packages/merlin/io/dataset.py:253: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
***** Running training *****
  Num examples = 107520
  Num Epochs = 10
  Instantaneous batch size per device = 448
  Total train batch size (w. parallel, distributed & accumulation) = 448
  Gradient Accumulation steps = 1
  Total optimization steps = 2400


********************
Launch training for day 29 are:
********************



Step,Training Loss
200,6.527700
400,6.292300
600,6.174300
800,6.064500
1000,6.026400
1200,5.953800
1400,5.897900
1600,5.854200
1800,5.828700
2000,5.803500


Saving model checkpoint to ./gru/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./gru/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./gru/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./gru/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




********************
Eval results for day 30 are:	

********************

 eval_/loss = 6.6362175941467285
 eval_/next-item/ndcg_at_10 = 0.1758965700864792
 eval_/next-item/ndcg_at_20 = 0.20357991755008698
 eval_/next-item/recall_at_10 = 0.3126915991306305
 eval_/next-item/recall_at_20 = 0.42205703258514404
 eval_runtime = 2.0994
 eval_samples_per_second = 6340.719
 eval_steps_per_second = 12.384


In [ ]:
with open("/content/drive/MyDrive/dataset_rees46/results.txt", 'a') as f: 
    f.write('\n')
    f.write('GRU accuracy results:')
    f.write('\n')
    for key, value in  model.compute_metrics().items(): 
        f.write('%s: %s' % (key, value.item()))

In [ ]:
print("Results:")
for key, value in  model.compute_metrics().items(): 
  print('\n%s: %s ' % (key, value.item()))

Results:
next-item/ndcg_at_10: 0.1758965700864792 
next-item/ndcg_at_20: 0.20357991755008698 
next-item/recall_at_10: 0.3126915991306305 
next-item/recall_at_20: 0.42205703258514404 
